In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_excel('data/sample - default of credit card clients.xls', skiprows=1, index_col='ID')
target_column = 'default payment next month'
features = df.drop(target_column, axis=1)
targets = df[target_column]
train_x, test_x, train_y, test_y = train_test_split(features, targets, stratify=targets, random_state=42)
scaler = StandardScaler()
scaled_train_x = scaler.fit_transform(train_x)
scaled_test_x = scaler.transform(test_x)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,,,,
0,30000,1,2,2,25,0,0,0,0,0,0,8864,10062,11581,12580,13716,14828,1500,2000,1500,1500,1500,2000,0
1,150000,2,1,2,26,0,0,0,0,0,0,136736,125651,116684,101581,77741,77264,4486,4235,3161,2647,2669,2669,0
2,70000,2,3,1,32,0,0,0,0,0,0,70122,69080,68530,69753,70111,70212,2431,3112,3000,2438,2500,2554,0
3,130000,1,3,2,49,0,0,0,0,0,-1,20678,18956,16172,16898,11236,6944,1610,1808,7014,27,7011,4408,0
4,50000,2,2,2,36,0,0,0,0,0,2,94228,47635,42361,19574,20295,19439,2000,1500,1000,1800,0,1000,1


In [13]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
lsvc.fit(scaled_train_x, train_y)
print(lsvc.score(scaled_train_x, train_y))
print(lsvc.score(scaled_test_x, test_y))

0.7964444444444444
0.7872


In [14]:
targets.value_counts(normalize=True)

0    0.783067
1    0.216933
Name: default payment next month, dtype: float64

In [15]:
from sklearn.svm import SVC

svc = SVC(probability=True)
svc.fit(scaled_train_x, train_y)
print(svc.score(scaled_train_x, train_y))
print(svc.score(scaled_test_x, test_y))

0.8272
0.8181333333333334


In [16]:
svc.predict_proba(scaled_test_x)

array([[0.85734362, 0.14265638],
       [0.41252091, 0.58747909],
       [0.1870021 , 0.8129979 ],
       ...,
       [0.8402008 , 0.1597992 ],
       [0.88203812, 0.11796188],
       [0.84144853, 0.15855147]])

In [25]:
from pycaret.distributions import UniformDistribution
from pycaret.classification import setup, create_model, tune_model

clf_setup = setup(data=df, target='default payment next month', normalize=True)
lsvc = create_model(LinearSVC)

,Description,Value
0,session_id,7350
1,Target,default payment next month
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(7500, 24)"
5,Missing Values,False
6,Numeric Features,14
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:13:30
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,SVM - Radial Kernel


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [26]:
tuned_lsvc = tune_model(lsvc, search_library='scikit-optimize', custom_grid={"C": UniformDistribution(0, 50)})

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8248,0.0000,0.3772,0.6719,0.4831,0.3875,0.4109
1,0.8324,0.0000,0.3772,0.7167,0.4943,0.4052,0.4352
2,0.7962,0.0000,0.3070,0.5556,0.3955,0.2850,0.3031
3,0.8343,0.0000,0.3860,0.7213,0.5029,0.4142,0.4434
4,0.8324,0.0000,0.3421,0.7500,0.4699,0.3864,0.4285
5,0.8190,0.0000,0.3333,0.6667,0.4444,0.3504,0.3805
6,0.8133,0.0000,0.3158,0.6429,0.4235,0.3273,0.3568
7,0.8152,0.0000,0.2982,0.6667,0.4121,0.3210,0.3576
8,0.8095,0.0000,0.2807,0.6400,0.3902,0.2972,0.3327
9,0.8073,0.0000,0.2566,0.6304,0.3648,0.2742,0.3129


In [27]:
tuned_lsvc

LinearSVC(C=2.094927808261094, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [32]:
lsvc = create_model('svm')
tuned_lsvc = tune_model(lsvc, search_library='scikit-optimize')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8133,0.0000,0.2807,0.6667,0.3951,0.3057,0.3459
1,0.8248,0.0000,0.2895,0.7500,0.4177,0.3376,0.3909
2,0.8019,0.0000,0.2544,0.6042,0.3580,0.2632,0.2978
3,0.8324,0.0000,0.3246,0.7708,0.4568,0.3766,0.4260
4,0.8171,0.0000,0.2632,0.7143,0.3846,0.3031,0.3556
5,0.8286,0.0000,0.3246,0.7400,0.4512,0.3675,0.4114
6,0.8190,0.0000,0.2895,0.7021,0.4099,0.3243,0.3688
7,0.8190,0.0000,0.2719,0.7209,0.3949,0.3132,0.3650
8,0.8000,0.0000,0.2193,0.6098,0.3226,0.2347,0.2772
9,0.8111,0.0000,0.2301,0.6842,0.3444,0.2645,0.3186


In [37]:
lsvc

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=7350, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [38]:
rbfsvc = create_model('rbfsvm')
tuned_rfbsvc = tune_model(rbfsvc, search_library='scikit-optimize')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8171,0.7455,0.3070,0.6731,0.4217,0.3306,0.3667
1,0.8286,0.7407,0.3246,0.7400,0.4512,0.3675,0.4114
2,0.8000,0.6907,0.2895,0.5789,0.3860,0.2820,0.3062
3,0.8324,0.7458,0.3596,0.7321,0.4824,0.3959,0.4316
4,0.8229,0.7115,0.2982,0.7234,0.4224,0.3385,0.3850
5,0.8229,0.7230,0.3509,0.6780,0.4624,0.3690,0.3977
6,0.8133,0.6785,0.2982,0.6538,0.4096,0.3167,0.3512
7,0.8190,0.7116,0.2982,0.6939,0.4172,0.3297,0.3710
8,0.8057,0.6508,0.2544,0.6304,0.3625,0.2715,0.3106
9,0.8015,0.6855,0.2301,0.6047,0.3333,0.2434,0.2828
